**Table of contents**<a id='toc0_'></a>    
- [**Introduction**](#toc1_)    
  - [Context](#toc1_1_)    
  - [Objectives](#toc1_2_)    
  - [Requirements](#toc1_3_)    
  - [Dataset](#toc1_4_)    
- [**First steps**](#toc2_)    
  - [Importing libraries](#toc2_1_)    
  - [Loading the dataset](#toc2_2_)    
  - [Data Overview](#toc2_3_)    
    - [Data description](#toc2_3_1_)    
    - [Data types](#toc2_3_2_)    
    - [Duplicate values](#toc2_3_3_)    
    - [Unique values / Missing values](#toc2_3_4_)    
  - [Interim summary](#toc2_4_)    
- [**Data splitting**](#toc3_)    
- [**Exploratory Data Analysis**](#toc4_)    
  - [Sidenote](#toc4_1_)    
  - [Target column distribution](#toc4_2_)    
  - [Numerical features and Outliers](#toc4_3_)    
  - [Categorical features](#toc4_4_)    
  - [Correlations](#toc4_5_)    
    - [Numerical features](#toc4_5_1_)    
    - [Categorical Association](#toc4_5_2_)    
      - [Upsampling](#toc4_5_2_1_)    
      - [Categorical Point Plots](#toc4_5_2_2_)    
    - [Numeric - Categorical Correlation](#toc4_5_3_)    
  - [Interim summary](#toc4_6_)    
- [**Statistical Inference**](#toc5_)    
  - [Difference in means](#toc5_1_)    
    - [Normality test](#toc5_1_1_)    
    - [Comparing Variance](#toc5_1_2_)    
    - [Welch's t-test](#toc5_1_3_)    
  - [Difference in proportions](#toc5_2_)    
    - [Chi-square test](#toc5_2_1_)    
  - [Interim summary](#toc5_3_)    
- [**Machine Learning Models**](#toc6_)    
  - [Feature Engineering](#toc6_1_)    
  - [Preprocessing pipeline](#toc6_2_)    
  - [Model Selection Metrics](#toc6_3_)    
  - [Benchmark Models](#toc6_4_)    
  - [Data Imbalance](#toc6_5_)    
    - [Oversampling](#toc6_5_1_)    
    - [Ensambling / Balanced Bagging](#toc6_5_2_)    
  - [Feature Selection](#toc6_6_)    
    - [K-Best](#toc6_6_1_)    
    - [Sequential Feature Selection](#toc6_6_2_)    
  - [Final Model](#toc6_7_)    
    - [Voting Classifier](#toc6_7_1_)    
- [**Final Model - Test Set**](#toc7_)    
- [**Summary**](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[**Introduction**](#toc0_)

## <a id='toc1_4_'></a>[Dataset](#toc0_)

[Stroke Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)

According to the World Health Organization (WHO) stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths.

This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.

Attribute Information:
1) id: unique identifier
2) gender: "Male", "Female" or "Other"
3) age: age of the patient
4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6) ever_married: "No" or "Yes"
7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8) Residence_type: "Rural" or "Urban"
9) avg_glucose_level: average glucose level in blood
10) bmi: body mass index
11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
12) stroke: 1 if the patient had a stroke or 0 if not

*Note: "Unknown" in smoking_status means that the information is unavailable for this patient


# <a id='toc2_'></a>[**First steps**](#toc0_)

This chapter for focuses on all the necessary steps to start the project. The required libraries will be imported, default values (if any) will be set, the dataset will be loaded, and the first exploratory data analysis will be performed.

## <a id='toc2_1_'></a>[Importing libraries](#toc0_)

The following libraries will be used in this project:
- polars: to work with dataframes.
- numpy: to work with numerical operations.
- plotly: to create interactive plots.
- scikit-learn: to create machine learning models.
- scipy: to perform statistical tests.
- imbalance-learn: to handle imbalanced datasets.
- dill: to serialize models.

Libraries and code will be wrapped in custom modules to make the code more readable and reusable, so only the necessary functions will be exposed to the user.

Custom modules:
 - "custom_modules.custom_functions": contains custom functions that wrap the libraries and code.
 - "custom_modules.constants": contains constants used in the project.
 - "custom_modules.custom_transformers": contains custom transformers to be used in the pipelines.
 - "custom_modules.custom_pickler": contains functions to serialize and deserialize objects.

In [ ]:
# custom modules
from custom_modules.custom_functions import load_dataset, get_unique_values, combine_data, upsample_minority_class
from custom_modules.custom_functions import plot_numerical_outliers, plot_bar_data, plot_points_subplots, plot_correlation_heatmap, plot_numeric_scatter, plot_catplot
from custom_modules.custom_functions import plot_violin, plot_precision_recall_curve
from custom_modules.custom_functions import continous_continous_correlation_matrix, categorical_categorical_correlation_matrix, plot_continous_categorical_correlation_matrix
from custom_modules.custom_functions import shapiro_wilk_normality_test_matrix, levene_variance_test, t_test, means_difference_and_ci, chi_square_test_matrix

from custom_modules.custom_functions import build_prep_pipeline, build_model_list, rebuild_balanced_models
from custom_modules.custom_functions import evaluate_model, model_score_matrix, evaluate_confusion_matrix
from custom_modules.custom_functions import select_k_best, evaluate_k_best, evaluate_sfs
from custom_modules.custom_functions import build_final_models_dict, hypertune_models

from custom_modules.custom_pickler import DillWrapper

from custom_modules.constants import RNG

# base modules
import polars as pl
import numpy as np

# graphing
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

## <a id='toc2_2_'></a>[Loading the dataset](#toc0_)

This project uses the "Stroke Prediction Dataset" dataset, which is available on Kaggle. Function will check if file is available and load it into a Polars DataFrame. If the file is not available, it will be downloaded from the Kaggle API.

In [2]:
df = load_dataset()

Dataset already downloaded


In [ ]:
df.head()

id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
i64,str,f64,i64,i64,str,str,str,f64,str,str,i64
9046,"""Male""",67.0,0,1,"""Yes""","""Private""","""Urban""",228.69,"""36.6""","""formerly smoked""",1
51676,"""Female""",61.0,0,0,"""Yes""","""Self-employed""","""Rural""",202.21,"""N/A""","""never smoked""",1
31112,"""Male""",80.0,0,1,"""Yes""","""Private""","""Rural""",105.92,"""32.5""","""never smoked""",1
60182,"""Female""",49.0,0,0,"""Yes""","""Private""","""Urban""",171.23,"""34.4""","""smokes""",1
1665,"""Female""",79.0,1,0,"""Yes""","""Self-employed""","""Rural""",174.12,"""24""","""never smoked""",1


## <a id='toc2_3_'></a>[Data Overview](#toc0_)

The first step is to get an overview of the data. 

The following information will be displayed:
- Data description.
- Data types.
- Missing values.
- Unique values.
- Duplicates.

### <a id='toc2_3_1_'></a>[Data description](#toc0_)

The data description will be displayed to give an overview of the dataset.

In [4]:
df.describe()

statistic,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
str,f64,str,f64,f64,f64,str,str,str,f64,str,str,f64
"""count""",5110.0,"""5110""",5110.0,5110.0,5110.0,"""5110""","""5110""","""5110""",5110.0,"""5110""","""5110""",5110.0
"""null_count""",0.0,"""0""",0.0,0.0,0.0,"""0""","""0""","""0""",0.0,"""0""","""0""",0.0
"""mean""",36517.829354,null,43.226614,0.097456,0.054012,null,null,null,106.147677,null,null,0.048728
"""std""",21161.721625,null,22.612647,0.296607,0.226063,null,null,null,45.28356,null,null,0.21532
"""min""",67.0,"""Female""",0.08,0.0,0.0,"""No""","""Govt_job""","""Rural""",55.12,"""10.3""","""Unknown""",0.0
"""25%""",17740.0,null,25.0,0.0,0.0,null,null,null,77.24,null,null,0.0
"""50%""",36942.0,null,45.0,0.0,0.0,null,null,null,91.89,null,null,0.0
"""75%""",54695.0,null,61.0,0.0,0.0,null,null,null,114.09,null,null,0.0
"""max""",72940.0,"""Other""",82.0,1.0,1.0,"""Yes""","""children""","""Urban""",271.74,"""N/A""","""smokes""",1.0


From the description we can see that the dataset has 5110 rows for 12 features. While describe method shows null_count for each column as 0, the head method shows that there are missing values at least in bmi column. This is possibly due to it being a string representation "N/A". These missing values will be handled in scikit-learn pipelines, with imputation strategies.

### <a id='toc2_3_2_'></a>[Data types](#toc0_)

The data types of each column will be displayed.

In [5]:
df.schema

Schema([('id', Int64),
        ('gender', String),
        ('age', Float64),
        ('hypertension', Int64),
        ('heart_disease', Int64),
        ('ever_married', String),
        ('work_type', String),
        ('Residence_type', String),
        ('avg_glucose_level', Float64),
        ('bmi', String),
        ('smoking_status', String),
        ('stroke', Int64)])

Again some steps can be taken to improve the data types. For example removing redundancy in column types, such as converting "age" to integer type (int8, assuming that the age is full number, and floats are outliers). As well working on converting binary columns to boolean type, string to category type, and so on.

All of this will be done in scikit-learn pipeline steps.

### <a id='toc2_3_3_'></a>[Duplicate values](#toc0_)

The dataset will be checked for duplicate values.

In [6]:
df.is_duplicated().sum()

0

Since data is collected from patients, with each having an unique id, there should be no duplicate values and duplicate count supported this.

In further steps, the id column will be dropped, as it is not a feature that can be used in the model.

### <a id='toc2_3_4_'></a>[Unique values / Missing values](#toc0_)

The unique values of each column will be displayed to get an overview of the data, as well as checking for "N/A" values.

In [7]:
with pl.Config(tbl_rows=15, tbl_width_chars=100, fmt_str_lengths=100):
    display(get_unique_values(df))

Column,Column Type,Unique Values Count,Unique Values,Contains N/A
str,object,i64,str,bool
"""id""",Int64,5110,"""More than 5 unique values""",false
"""gender""",String,3,"""Female, Other, Male""",false
"""age""",Float64,104,"""More than 5 unique values""",false
"""hypertension""",Int64,2,"""0, 1""",false
"""heart_disease""",Int64,2,"""0, 1""",false
"""ever_married""",String,2,"""No, Yes""",false
"""work_type""",String,5,"""Self-employed, Private, children, Govt_job, Never_worked""",false
"""Residence_type""",String,2,"""Urban, Rural""",false
"""avg_glucose_level""",Float64,3979,"""More than 5 unique values""",false


Again this confirms that no duplicates should be present in the dataset, due to unique values in "id" column being consistent with the number of rows.

"age" column is suspicious, as it has 104 unique values, which is not expected for age, but not unheard of. Previous steps showed that at least some float values are present

Everything else is as expected, with binary columns having 2 unique values, and categorical columns having more than 2 unique values.

## <a id='toc2_4_'></a>[Interim summary](#toc0_)

The first steps of the project have been completed. The necessary libraries have been imported, the dataset has been loaded, and the first exploratory data analysis has been performed. 

The dataset contains missing values which will be handled in scikit-learn pipelines using imputation methods.

The data types will be improved in scikit-learn pipelines, by converting columns to more appropriate types e.g. "ever_married" to boolean, "work_type" to category, etc.

As well there is a possibility of invalid values in "age" column, which will be investigated further.

# <a id='toc3_'></a>[**Data splitting**](#toc0_)

To avoid data leakage, biases, and overfitting, the data will be split into training and testing sets. The training set will be used to train the model, while the testing set will be used to evaluate the model. It is done so early in the project to avoid any potential biases in the exploratory data analysis.

Data is split into 80% training and 20% testing sets.

Random state is set to 42 to ensure reproducibility.

Stratification is used to ensure that the distribution of the target variable is the same in both training and testing sets.

"id" column is dropped, as it is not a feature that can be used in the model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["stroke", "id"]), df["stroke"], test_size=0.2, random_state=RNG, stratify=df["stroke"])
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (4088, 10)
X_test shape: (1022, 10)


# <a id='toc4_'></a>[**Exploratory Data Analysis**](#toc0_)

This chapter focuses on the exploratory data analysis. The data will be analyzed to find patterns, correlations, and other relations between variables. This will help to understand the data better and to make better decisions for the machine learning model.

## <a id='toc4_1_'></a>[Sidenote](#toc0_)

Training set will be recombined with the target variable for EDA, as it is not necessary to split the target variable from the training set for EDA. And binary columns answers will be converted to Yes/No for better readability (including "stroke").

In [ ]:
df = combine_data(X_train, y_train, ["hypertension", "heart_disease", "stroke"])
df.head()   

gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
str,f64,str,str,str,str,str,f64,str,str,str
"""Female""",48.0,"""No""","""No""","""Yes""","""Private""","""Urban""",69.21,"""33.1""","""never smoked""","""No"""
"""Male""",15.0,"""No""","""No""","""No""","""Private""","""Rural""",122.25,"""21""","""never smoked""","""No"""
"""Female""",67.0,"""No""","""No""","""Yes""","""Self-employed""","""Rural""",110.42,"""24.9""","""never smoked""","""No"""
"""Male""",44.0,"""No""","""No""","""Yes""","""Private""","""Urban""",65.41,"""24.8""","""smokes""","""No"""
"""Male""",14.0,"""No""","""No""","""No""","""Govt_job""","""Urban""",82.34,"""31.6""","""Unknown""","""No"""


## <a id='toc4_2_'></a>[Target column distribution](#toc0_)

The distribution of the target column will be checked to see how balanced the dataset is. Due to stratification used, the distribution should be similar to the original dataset.

In [10]:
df["stroke"].value_counts()

stroke,count
str,u32
"""Yes""",199
"""No""",3889


As suspected, the dataset is imbalanced, with the majority of patients not having a stroke. This will be taken into account when building the machine learning model.

## <a id='toc4_3_'></a>[Numerical features and Outliers](#toc0_)

This section focuses on numerical features and their outliers to try and spot any patterns or anomalies in the data.

This is done by creating box plots for each numerical feature.

In [11]:
plot_numerical_outliers(df, ["age", "avg_glucose_level", "bmi"])

One feature where two groups differ significantly (at least visually) is "age". Patients who had a stroke seem to be older than those who did not have a stroke. This is a good sign that the feature might be useful in predicting the target variable.

There doesn't seem to be a significant difference in other numerical features, but this will be further investigated in the next steps.

In [12]:
(df["age"] < 1).value_counts()

age,count
bool,u32
true,34
false,4054


There are 34 patients with age being lower than 1, which could suggest infants or an issue with data collection. 

There is as well a patient age 1.32 who had a stroke. This could be a data collection issue, or a valid data point, either way as far as strokes are concerned, it is an outlier. 

As data will be normalized in scikit-learn pipelines, this should not be an issue.

## <a id='toc4_4_'></a>[Categorical features](#toc0_)

This section focuses on categorical features and their distribution to try and spot any patterns or anomalies in the data.

In [13]:
plot_bar_data(df, cat_columns=["gender", "hypertension", "heart_disease", "ever_married", "work_type", "Residence_type", "smoking_status"], target="stroke")

It is hard to tell if there is any significant difference in the distribution of categorical features between patients who had a stroke and those who did not.

In [14]:
plot_points_subplots(df, "stroke", "Yes", 0.95, ["gender", 
                                                 "hypertension", 
                                                 "heart_disease", 
                                                 "ever_married", 
                                                 "work_type", 
                                                 "Residence_type", 
                                                 "smoking_status"])

When looking over the percentages of patients who had a stroke, it seems that patients who have hypertension, heart disease, have ever married, smoked in the past or are self-employed are more likely to have a stroke. This perhaps is a good sign that these features might be useful in predicting the target variable.

The question here is how significant these differences are, and if they are significant enough to be used in the model. As well as how these features interact with each other, for example ever married - does this mean that married people are more likely to have a stroke, or are there some more underlying factors like age at play.

## <a id='toc4_5_'></a>[Correlations](#toc0_)

This section focuses on correlations between features and the target variable to try and spot any patterns or anomalies in the data.

### <a id='toc4_5_1_'></a>[Numerical features](#toc0_)

The correlation matrix will be displayed to show the correlation between numerical features.

To plot rough correlation between numerical features the "N/A" values will be imputed with the mean value of the column.

In [15]:
mean_bmi = df.filter(pl.col("bmi") != "N/A")["bmi"].cast(pl.Float32).mean()
filled_bmi = df.select(pl.col("bmi").replace({"N/A": str(mean_bmi)})).cast(pl.Float32)["bmi"]
df = df.select(pl.all().exclude("bmi")).with_columns(filled_bmi)

In [16]:
plot_correlation_heatmap(continous_continous_correlation_matrix(df["age", 'avg_glucose_level', "bmi"]))

There is low-moderate correlation between age and bmi features, no other significant correlations are present.

In [17]:
plot_numeric_scatter(df, ["bmi", "age"], "stroke")

From the scatter plot it is visible that there is some relationship between age and bmi.

From the scatter plot it also visible that a higher amount of patients who had a stroke have higher bmi values, with anything above 25 being considered overweight.

### <a id='toc4_5_2_'></a>[Categorical Association](#toc0_)

The association between categorical features and the target variable will be displayed.


In [18]:
plot_correlation_heatmap(data=categorical_categorical_correlation_matrix(df.select(pl.col(pl.String))),
                         title="Categorical-Categorical Correlation Matrix",
                         color_scale="Blues",
                         zmin=0,
                         sub_title="Cramer's V")


There doesn't seem to be a significant association between categorical features and the target variable. 

There is some association between ever_married and work_type features, as well as these two features and smoking_status.

Another point to consider is that the Cramer's V test is not the best test for this dataset, as it is unbalanced, and the test is more suited for balanced datasets. Perhaps upsampling or downsampling could be used to balance the dataset, or another test could be used. 

It this case it is not clear if the features are associated with the target variable.

#### <a id='toc4_5_2_1_'></a>[Upsampling](#toc0_)

In this case a simple resampling technique will be used to balance the dataset. Which will copy the rows of the minority class to match the majority class.

Alternatively, SMOTE (Synthetic Minority Over-sampling Technique) could be used to generate synthetic samples for the minority class. However, this technique requires additional libraries to be installed, and it is not clear if it is necessary in this case.

Upsampling like this is only done for the EDA, because if used in the model, it could lead to overfitting.

For machine learning models, the dataset will be considered as is, but more robust ideas will be considered to handle the imbalance,- like loss functions or appropriate classfiers.

In [19]:
df_upsampled = upsample_minority_class(df, "stroke", "Yes")
df_upsampled["stroke"].value_counts()

stroke,count
str,u32
"""Yes""",3889
"""No""",3889


In [20]:
plot_correlation_heatmap(data=categorical_categorical_correlation_matrix(df_upsampled.select(pl.col(pl.String))),
                         title="Categorical-Categorical Correlation Matrix (Upsampled)",
                         color_scale="Blues",
                         zmin=0,
                         sub_title="Cramer's V")

Upon upsampling, the association between categorical features and the target variable is more visible. However, the association is still not significant to say that the features are useful in predicting the target variable.

#### <a id='toc4_5_2_2_'></a>[Categorical Point Plots](#toc0_)

The point plots will be displayed to show the relationship between categorical features and the target variable.

In [21]:
plot_catplot(df, ["work_type", "ever_married"], "stroke", "Yes", 0.95)

Again the graphs suggest that if a patient has ever married, they are more likely to have a stroke. Most of the difference comes from "Private" work type, where the difference between ever married patients is the highest in terms of stroke percentage. 

This could be due to age, as older people are more likely to have a stroke, and older people are more likely to be married. This will be investigated further down the line. 

In [22]:
plot_catplot(df, ["smoking_status", "ever_married"], "stroke", "Yes", 0.95)

The data suggests that if a patient has ever smoked and is married, they are more likely to have a stroke. 

Again it is yet unclear how being married leads to a higher chance of having a stroke, and if there are any underlying factors at play.

### <a id='toc4_5_3_'></a>[Numeric - Categorical Correlation](#toc0_)

The correlation between numerical and categorical features will be displayed.

In [23]:
plot_continous_categorical_correlation_matrix(data=df_upsampled,
                                              categorical_columns=df.select(pl.col(pl.String)).columns,
                                              continous_columns=df.select(pl.all().exclude(pl.String)).columns,
                                              title="Correlation Matrix (Upsampled)")

If using upsampling, the correlation between numerical and categorical features is more visible. And there is medium-to-high correlation between age and stroke/ever_married features.

If using the original dataset, the correlation between numerical and categorical features is smaller in value but still present.

So this essentially confirms the previous findings, that age is a significant feature in predicting the target variable. Another point to consider is the correlation between age and ever_married, previous findings suggest that if a patient is ever married, they are more likely to have a stroke. This could be due to age, as older people are more likely to be married, and older people are more likely to have a stroke.

In [24]:
plot_violin(df, "stroke", ["ever_married", "age"])

It is unclear if ever_married is a significant feature in predicting the target variable, it looks like it is just a proxy for age.

In [25]:
plot_violin(df, "stroke", ["stroke", "avg_glucose_level"])

As far as avg_glucose_level and stroke feature is concerned, it is hard to tell if there is any significant correlation between the two features. The median is slightly higher for patients who had a stroke, but the difference is not significant.

## <a id='toc4_6_'></a>[Interim summary](#toc0_)

The exploratory data analysis has shown that there is some relationship between age and the target variable. Patients who had a stroke seem to be older than those who did not have a stroke. This is a good sign that the feature might be useful in predicting the target variable.

As far as other features are concerned, it is hard to tell if they are significant in predicting the target variable. There is some association between categorical features and the target variable, but it is not significant enough to say that the features are useful in predicting the target variable.

The dataset is imbalanced, with the majority of patients not having a stroke. This will be taken into account when building the machine learning model.

# <a id='toc5_'></a>[**Statistical Inference**](#toc0_)

This chapter focuses on statistical inference. The target population will be explored, multiple statistical hypotheses will be formed and tested.

Significance level: 0.05

Confidence level: 95%

## <a id='toc5_1_'></a>[Difference in means](#toc0_)

The difference in means between patients who had a stroke and those who did not have a stroke will be tested for numerical features. Assumption here is that if there is a significant difference in means, the feature is useful in predicting the target variable.

Features that will be tested:
- age
- avg_glucose_level
- bmi

### <a id='toc5_1_1_'></a>[Normality test](#toc0_)

One of the assumptions of the t-test is that the data is normally distributed. The Shapiro-Wilk test will be used to test for normality.

Null hypothesis: the data is normally distributed.

Alternative hypothesis: the data is not normally distributed.

Significance level: 0.05

In [26]:
shapiro_wilk_normality_test_matrix(df["age", "bmi", "avg_glucose_level", "stroke"], "stroke")

stroke,Column,Statistic,PValue
str,str,f64,f64
"""Yes""","""age""",0.8917,8.2888e-11
"""Yes""","""bmi""",0.92583,1.7014e-8
"""Yes""","""avg_glucose_level""",0.870416,5.1292e-12
"""No""","""age""",0.97006,9.1727e-28
"""No""","""bmi""",0.94719,2.9402e-35
"""No""","""avg_glucose_level""",0.805585,4.9169e-56


In [27]:
df["stroke"].value_counts()

stroke,count
str,u32
"""No""",3889
"""Yes""",199


None of the numerical features are normally distributed, which suggests that the t-test might not be the best test to use in this case. However, the t-test is robust to violations of normality, as well as the consideration of Central Limit Theorem can come into play, which states that the distribution of sample means approaches a normal distribution as the sample size gets larger.

Considering the sample size for both groups is large enough, the t-test can be used (199 samples for having a stroke, 3889 samples for not having a stroke), again due to the Central Limit Theorem.

### <a id='toc5_1_2_'></a>[Comparing Variance](#toc0_)

T-test assumes that the variances of the two groups are equal. The Levene's test will be used to test for equal variances due to features not being normally distributed.

Null hypothesis: the variances of the two groups are equal.

Alternative hypothesis: the variances of the two groups are not equal.

In [28]:
levene_variance_test(df, ["age", "bmi", "avg_glucose_level", "stroke"], "stroke", ["Yes", "No"])

Column,Statistic,PValue
str,f64,f64
"""age""",111.143058,1.1721e-25
"""bmi""",24.08249,9.5893e-7
"""avg_glucose_level""",76.218291,3.6413e-18


Levene's test results suggest that the variances of the two groups are not equal. Since the variances are not equal, the Welch's t-test will be used to test for the difference in means.

### <a id='toc5_1_3_'></a>[Welch's t-test](#toc0_)

The Welch's t-test will be used to test for the difference in means between patients who had a stroke and those who did not have a stroke.

Null hypothesis: there is no difference in means between patients who had a stroke and those who did not have a stroke.

Alternative hypothesis: there is a difference in means between patients who had a stroke and those who did not have a stroke.

In [29]:
t_test(df, ["age", "bmi", "avg_glucose_level", "stroke"], "stroke", ["Yes", "No"])

Column,Statistic,PValue
str,f64,f64
"""age""",27.931077,2.4446e-82
"""bmi""",3.323724,0.001029
"""avg_glucose_level""",5.908952,1.3890e-8


In all cases, the p-value is less than the significance level, which suggests that there is statistically significant difference in means between patients who had a stroke and those who did not have a stroke. This is a good sign that the features might be useful in predicting the target variable.

In [30]:
means_difference_and_ci(df, ["age", "bmi", "avg_glucose_level", "stroke"], "stroke", ["Yes", "No"], 0.95)

Column,Mean_A (Yes),Mean_B (No),Difference,CI_Lower,CI_Upper
str,f64,f64,f64,f64,f64
"""age""",67.664925,42.109262,25.555663,23.754471,27.356854
"""bmi""",30.283521,28.85252,1.431001,0.582815,2.279186
"""avg_glucose_level""",131.389799,105.034202,26.355597,17.562494,35.148701


In all the cases, the mean of the patients who had a stroke is higher than the mean of the patients who did not have a stroke. This is consistent with the exploratory data analysis, where it was shown that patients who had a stroke seem to be older than those who did not have a stroke. Confidence intervals also support this, as they do not contain 0.

## <a id='toc5_2_'></a>[Difference in proportions](#toc0_)

The difference in proportions between patients who had a stroke and those who did not have a stroke will be tested for categorical features. Assumption here is that if there is a significant difference in proportions, the feature is useful in predicting the target variable.

Features that will be tested:
- gender
- hypertension
- heart_disease
- ever_married
- work_type
- Residence_type
- smoking_status

### <a id='toc5_2_1_'></a>[Chi-square test](#toc0_)

The Chi-square test for Homogeneity will be used to test for the difference in proportions between patients who had a stroke and those who did not have a stroke.

Null hypothesis: there is no difference in proportions between patients who had a stroke and those who did not have a stroke.

Alternative hypothesis: there is a difference in proportions between patients who had a stroke and those who did not have a stroke.

In [31]:
chi_square_test_matrix(df.select(pl.col(pl.String)), target_col="stroke")

Feature,Chi-Square,P-Value
str,f64,f64
"""gender""",0.515003,0.772981
"""hypertension""",70.357308,4.9479e-17
"""heart_disease""",63.232778,1.8367e-15
"""ever_married""",45.741062,1.3496e-11
"""work_type""",43.682652,7.4671e-9
"""Residence_type""",0.97228,0.324112
"""smoking_status""",23.063909,0.000039


Some of the values have different distributions, which suggests that there is a significant difference in proportions between patients who had a stroke and those who did not have a stroke,- this should help in predicting the target variable.

On the other hand it is hard to tell how the features interact with each other or the target variable. For example how does being married lead to a higher chance of having a stroke, or are there some more underlying factors at play. Current idea is that age is a significant factor in predicting the target variable, and other features are just proxies (confounding) for age.  

## <a id='toc5_3_'></a>[Interim summary](#toc0_)

The statistical inference has shown that there is a significant difference between patients who had a stroke and those who did not have a stroke. This is true for both numerical and categorical features. Although for categorical features it is hard to tell how the features interact with the target variable.

For numerical features, the difference in means is significant, which suggests that if values are higher, the patient is more likely to have a stroke. This is consistent with the exploratory data analysis, where it was shown that patients who had a stroke seem to be older than those who did not have a stroke.


# <a id='toc6_'></a>[**Machine Learning Models**](#toc0_)

This chapter focuses on building machine learning models. Various machine learning models will be applied to predict the "stroke" column using all other features. This will include hyperparameter tuning, model ensembling, the analysis of model selection, and other methods.

Key considerations:
- The dataset is imbalanced, with the majority of patients not having a stroke. This will be taken into account when building the machine learning model.
    - Loss functions or appropriate classifiers can be considered to handle the imbalance.
    - SMOTE (Synthetic Minority Over-sampling Technique) could be used to generate synthetic samples for the minority class.
- The dataset contains missing values in the "bmi" column. These missing values can be handled in scikit-learn pipelines using imputation methods.
- Normalization, this can be done in scikit-learn pipelines.

## <a id='toc6_1_'></a>[Feature Engineering](#toc0_)

Numeric binning will be used to convert numerical features to categorical features. This should help capture non-linear relationships between features and the target variable.
- age
    - < 18: teen
    - 19-39: adult
    - 40-59: middle-aged
    - \> 60 : senior
- bmi
    - < 19: underweight
    - 19-25: normal
    - 25-30: overweight
    - \> 30: obese
- avg_glucose_level
    - < 70: low
    - 70-100: normal
    - 100-125: pre-diabetic
    - \> 125: diabetic

Additional features:
- risk_score_all = age + bmi + avg_glucose_level + hypertension + heart_disease
- risk_score_numeric_no_age = bmi + avg_glucose_level
- glucose_bmi_ratio = avg_glucose_level / bmi
- risk_score_numeric = bmi + avg_glucose_level + age
- risk_score_heart = hypertension + heart_disease

These extra features are created to hopefully capture non-linear relationships between features and the target variable. And may be culled later on if they are not useful.

## <a id='toc6_2_'></a>[Preprocessing pipeline](#toc0_)

The preprocessing pipeline will be created to handle missing values, normalize data, and convert data types.

This pipeline step uses ColumnTransformer to apply different transformations to different columns. The pipeline will be used to handle missing values, normalize data, and convert data types.

Specifically:
- Numerical columns (age, avg_glucose_level, bmi):
    - Converts missing values to -1 as a placeholder for imputation and casts the column to float type. (bmi)
    - Imputes missing values using KNNImputer(-1 as missing value,- bmi).
    - Normalizes data using Normalizer.(range 0-1 - age, avg_glucose_level, bmi)
- Binary columns (hypertension, heart_disease, ever_married):
    - Imputes missing values using SimpleImputer(strategy="most_frequent").
    - Converts binary Yes/No answers to 1/0.
    - Converts binary columns to Int8 type (although it is not necessary, it is done for consistency).
- Categorical (gender, work_type, Residence_type, smoking_status):
    - Imputes missing values using SimpleImputer(strategy="most_frequent").
    - One-hot encodes categorical columns.
- Drops any other columns. (id)


In [32]:
prep_pipeline = build_prep_pipeline()
display(prep_pipeline)
df = build_prep_pipeline().fit_transform(X_train)
display(df.head())

Pipeline(steps=[('preprocessor',
                 FeaturePreprocessor(column_pipelines={'Residence_type': Pipeline(steps=[('cat_imputer',
                                                                                          NumericImputer(adj_missing_value=False,
                                                                                                         imputer='simple',
                                                                                                         strategy='most_frequent')),
                                                                                         ('one_hot_encoder',
                                                                                          OneHotEncoderWrapper()),
                                                                                         ('drop',
                                                                                          {'drop': 'Residence_type'})]),
                                                       'age': Pipeline(steps=[('imputer',
                                                                               NumericImputer()),
                                                                              ('binner',
                                                                               NumericB...
                                                       new_feature_name='glucose_bmi_ratio',
                                                       scaler_obj=NumericScaler())),
                                 ('risk_score_heart',
                                  BasicFeatureCombiner(columns=['heart_disease',
                                                                'hypertension'],
                                                       new_feature_name='risk_score_heart',
                                                       scaler_obj=NumericScaler())),
                                 ('risk_score_numeric',
                                  BasicFeatureCombiner(columns=['bmi',
                                                                'avg_glucose_level',
                                                                'age'],
                                                       new_feature_name='risk_score_numeric',
                                                       scaler_obj=NumericScaler()))]))])

age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,age_binned_adult,age_binned_middle_age,age_binned_senior,age_binned_teen,avg_glucose_level_binned_diabetic,avg_glucose_level_binned_low-normal,avg_glucose_level_binned_normal,avg_glucose_level_binned_pre-diabetic,bmi_binned_normal,bmi_binned_obese,bmi_binned_overweight,bmi_binned_underweight,risk_score_all,risk_score_no_age,glucose_bmi_ratio,risk_score_heart,risk_score_numeric
f64,i8,i8,i8,f64,f64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,f64,f64,f64,f64,f64
0.584961,0,0,1,0.065045,0.261168,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0.208145,0.206574,0.013943,0.0,0.370051
0.182129,0,0,0,0.309898,0.122566,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0.134614,0.28345,0.141548,0.0,0.239323
0.816895,0,0,1,0.255286,0.167239,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0.289527,0.276259,0.085456,0.0,0.514735
0.536133,0,0,1,0.047503,0.166094,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0.168118,0.125092,0.016011,0.0,0.298889
0.169922,0,0,0,0.125658,0.243986,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0.116013,0.237998,0.028832,0.0,0.206253


In [33]:
px.scatter(data_frame=df, x="bmi", y="age", color="risk_score_all", facet_col=y_train, title="BMI vs Age vs Risk Score")

With all the preprocessing steps in place, the data is ready to be used in machine learning models.

## <a id='toc6_3_'></a>[Model Selection Metrics](#toc0_)

Since the aim is to predict if a patient is likely to get a stroke, the amount of False Negatives (missed positives) should be minimized. This is because it is better to predict that a patient will have a stroke and be wrong, than to predict that a patient will not have a stroke and be wrong.

Due to this, the focus will be on the amount of True Positives and False Negatives (type II error). In this case, the recall score will be used as the main metric for model selection. The higher the recall score, the better the model is at predicting the target variable, and the lower the amount of False Negatives.

As a secondary metric, the F1 score will be used, as it is a good balance between precision and recall.


## <a id='toc6_4_'></a>[Benchmark Models](#toc0_)

Some different classifiers will be tested to see which one performs the best and to get a rough idea of well how the models perform.

In [34]:
result = DillWrapper(function=model_score_matrix, 
                     models=build_model_list(prep_pipeline), 
                     X=X_train, 
                     y=y_train, 
                     scoring_metric="recall").load(create_if_not_exist=True)
display(result[0].sort("mean_test_score", descending=True).head())
result[1].show()

Model (recall),mean_test_score,test_score_sd,Time Taken
str,f64,f64,f64
"""Gaussian Naive Bayes""",0.978885,0.019042,0.622694
"""Decision Tree""",0.197513,0.075409,0.7389
"""Extra Tree""",0.135205,0.053615,0.619105
"""Passive Aggressive""",0.118449,0.162241,0.669511
"""Gradient Boosting""",0.039141,0.039503,6.145291


In [35]:
DillWrapper(function=evaluate_confusion_matrix, 
            model_dict=build_model_list(prep_pipeline), 
            X_train=X_train, 
            y_train=y_train,
            n_splits=5, 
            n_repeats=2).load(create_if_not_exist=True)

Observation,Random Forest,Gradient Boosting,HistGradientBoosting,Logistic Regression,SGD Classifier,Ridge Classifier,Gaussian Naive Bayes,Passive Aggressive,KNNeighbors,SVC,Linear SVC,Decision Tree,Extra Tree
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Mean True Negatives""","""776.2 (0.95)""","""770.5 (0.94)""","""776.2 (0.95)""","""777.8 (0.95)""","""777.4 (0.95)""","""777.8 (0.95)""","""360.8 (0.44)""","""741.7 (0.91)""","""774.4 (0.95)""","""777.8 (0.95)""","""777.8 (0.95)""","""720.1 (0.88)""","""735.1 (0.9)"""
"""Mean False Positives""","""1.6 (0.0)""","""7.3 (0.01)""","""1.6 (0.0)""","""0.0 (0.0)""","""0.4 (0.0)""","""0.0 (0.0)""","""417.0 (0.51)""","""36.1 (0.04)""","""3.4 (0.0)""","""0.0 (0.0)""","""0.0 (0.0)""","""57.7 (0.07)""","""42.7 (0.05)"""
"""Mean False Negatives""","""39.4 (0.05)""","""38.3 (0.05)""","""39.5 (0.05)""","""39.8 (0.05)""","""39.8 (0.05)""","""39.8 (0.05)""","""0.9 (0.0)""","""34.0 (0.04)""","""39.7 (0.05)""","""39.8 (0.05)""","""39.8 (0.05)""","""30.8 (0.04)""","""34.1 (0.04)"""
"""Mean True Positives""","""0.4 (0.0)""","""1.5 (0.0)""","""0.3 (0.0)""","""0.0 (0.0)""","""0.0 (0.0)""","""0.0 (0.0)""","""38.9 (0.05)""","""5.8 (0.01)""","""0.1 (0.0)""","""0.0 (0.0)""","""0.0 (0.0)""","""9.0 (0.01)""","""5.7 (0.01)"""


A lot of models have very bad recall scores, most likely caused by the imbalance in the dataset, as none of the models can accurately predict the target variable. And most likely the models are predicting the majority class, hence high accuracy scores.

On the other hand Naive Bayes has almost perfect recall score, which at very least looks suspicious. An upon investigation, it seems that the model is predicting roughly 50/50 split between the two classes, which is not what is needed.

Possible models to use:
- Decision Tree - showed good recall and f1 scores. Overall a simple model that captures non-linear relationships between features and the target variable. Overall all tree-based models should work well, however, random forest and gradient boosting models did not perform well in this case.
- Passive Aggressive Classifier - showed good recall and f1 scores.

The next step should be to try and improve the imbalance in the dataset, which in turn should improve the recall score of the models. As well as try to figure out which features are redundant, and which features are useful in predicting the target variable.

From now on Decision Tree and Passive Aggressive Classifier will be used as benchmark models.


## <a id='toc6_5_'></a>[Data Imbalance](#toc0_)

This section focuses on handling the imbalance in the dataset.

Couple of possible solutions:
- Class weights/loss functions: assign higher weights to the minority class, or use loss functions that are more robust to imbalanced datasets.
- Oversampling: generate synthetic samples for the minority class.
- Ensambling: generate same ratio minority/majority (same minority sample, different majority sample) class samples for each model in the ensemble.

### <a id='toc6_5_1_'></a>[Oversampling](#toc0_)

SMOTE (Synthetic Minority Over-sampling Technique) will be used to generate synthetic samples for the minority class.

In [36]:
sm = SMOTE(random_state=RNG)
X_train_resampled, y_train_resampled = sm.fit_resample(df.to_numpy(), y_train)
X_train_resampled = pl.DataFrame(X_train_resampled, schema=df.schema)
print(f"New upsampled shape: {X_train_resampled.shape}")

New upsampled shape: (7778, 37)


In [37]:
result = DillWrapper(function=model_score_matrix, 
                     models=build_model_list(prep_pipeline), 
                     X=X_train_resampled, 
                     y=y_train_resampled, 
                     scoring_metric="recall").load(create_if_not_exist=True)
display(result[0].sort("mean_test_score", descending=True).head())
result[1].show()

Model (recall),mean_test_score,test_score_sd,Time Taken
str,f64,f64,f64
"""SVC""",0.852278,0.039045,7.081448
"""Logistic Regression""",0.820499,0.041136,0.34131
"""Ridge Classifier""",0.81857,0.063073,0.311283
"""Gradient Boosting""",0.802,0.065256,8.938499
"""SGD Classifier""",0.793608,0.122001,0.381346


In [38]:
DillWrapper(function=evaluate_confusion_matrix, 
            model_dict=build_model_list(prep_pipeline), 
            X_train=X_train_resampled, 
            y_train=y_train_resampled,
            n_splits=5, 
            n_repeats=2).load(create_if_not_exist=True)

Observation,Random Forest,Gradient Boosting,HistGradientBoosting,Logistic Regression,SGD Classifier,Ridge Classifier,Gaussian Naive Bayes,Passive Aggressive,KNNeighbors,SVC,Linear SVC,Decision Tree,Extra Tree
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Mean True Negatives""","""718.0 (0.46)""","""610.6 (0.39)""","""671.3 (0.43)""","""578.2 (0.37)""","""592.9 (0.38)""","""576.4 (0.37)""","""619.3 (0.4)""","""498.5 (0.32)""","""613.4 (0.39)""","""566.9 (0.36)""","""580.5 (0.37)""","""641.6 (0.41)""","""658.4 (0.42)"""
"""Mean False Positives""","""59.8 (0.04)""","""167.2 (0.11)""","""106.5 (0.07)""","""199.6 (0.13)""","""184.9 (0.12)""","""201.4 (0.13)""","""158.5 (0.1)""","""279.3 (0.18)""","""164.4 (0.11)""","""210.9 (0.14)""","""197.3 (0.13)""","""136.2 (0.09)""","""119.4 (0.08)"""
"""Mean False Negatives""","""416.9 (0.27)""","""175.0 (0.11)""","""192.9 (0.12)""","""141.2 (0.09)""","""180.9 (0.12)""","""142.3 (0.09)""","""317.0 (0.2)""","""161.8 (0.1)""","""210.6 (0.14)""","""118.2 (0.08)""","""184.2 (0.12)""","""408.5 (0.26)""","""367.6 (0.24)"""
"""Mean True Positives""","""360.9 (0.23)""","""602.8 (0.39)""","""584.9 (0.38)""","""636.6 (0.41)""","""596.9 (0.38)""","""635.5 (0.41)""","""460.8 (0.3)""","""616.0 (0.4)""","""567.2 (0.36)""","""659.6 (0.42)""","""593.6 (0.38)""","""369.3 (0.24)""","""410.2 (0.26)"""


Upsampling helps to balance the dataset, and the results are more visible in the confusion matrix. All the models are now more likely to predict the minority class. Currently the issue with this is False Negatives, counts of which are still high,- so the models are not predicting the minority class as well as they should.

Decision Tree performance dropped a little, as it doesn't stand out from the rest as much as it did before.

Passive Aggressive Classifier is still relatively good, though it's mean recall score is all over the place, with multiple outliers.

All other models benefited from upsampling, with their recall scores improving significantly.

Also the results can be improved by removing some of the features that are not useful in predicting the target variable, as currently there are possibly too many features in the dataset, leading to some of the models to overfitting.


### <a id='toc6_5_2_'></a>[Ensambling / Balanced Bagging](#toc0_)

BalancedBaggingClassifier should in theory address the imbalance in the dataset, as it generates same ratio minority/majority class samples for each model in the ensemble.


In [39]:
balanced_model_list = rebuild_balanced_models(prep_pipeline)

In [40]:
result = DillWrapper(function=model_score_matrix, 
                     models=balanced_model_list, 
                     X=X_train, 
                     y=y_train, 
                     scoring_metric="recall").load(create_if_not_exist=True)
display(result[0].sort("mean_test_score", descending=True).head())
result[1].show()

Model (recall),mean_test_score,test_score_sd,Time Taken
str,f64,f64,f64
"""Gaussian Naive Bayes""",0.979397,0.018561,0.826713
"""Logistic Regression""",0.827718,0.068697,1.012922
"""Ridge Classifier""",0.824167,0.073693,0.844768
"""Linear SVC""",0.810128,0.074935,0.862783
"""KNNeighbors""",0.77541,0.071764,0.909827


In [41]:
DillWrapper(function=evaluate_confusion_matrix, 
            model_dict=balanced_model_list, 
            X_train=X_train, 
            y_train=y_train,
            n_splits=5, 
            n_repeats=2).load(create_if_not_exist=True)

Observation,Random Forest,Gradient Boosting,HistGradientBoosting,Logistic Regression,SGD Classifier,Ridge Classifier,Gaussian Naive Bayes,Passive Aggressive,KNNeighbors,SVC,Linear SVC,Decision Tree,Extra Tree
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Mean True Negatives""","""584.6 (0.72)""","""599.4 (0.73)""","""605.4 (0.74)""","""537.3 (0.66)""","""538.7 (0.66)""","""543.1 (0.66)""","""358.7 (0.44)""","""596.3 (0.73)""","""529.3 (0.65)""","""554.8 (0.68)""","""549.7 (0.67)""","""652.1 (0.8)""","""628.3 (0.77)"""
"""Mean False Positives""","""193.2 (0.24)""","""178.4 (0.22)""","""172.4 (0.21)""","""240.5 (0.29)""","""239.1 (0.29)""","""234.7 (0.29)""","""419.1 (0.51)""","""181.5 (0.22)""","""248.5 (0.3)""","""223.0 (0.27)""","""228.1 (0.28)""","""125.7 (0.15)""","""149.5 (0.18)"""
"""Mean False Negatives""","""9.1 (0.01)""","""11.3 (0.01)""","""11.7 (0.01)""","""7.0 (0.01)""","""6.8 (0.01)""","""6.3 (0.01)""","""0.9 (0.0)""","""13.4 (0.02)""","""9.5 (0.01)""","""9.1 (0.01)""","""6.9 (0.01)""","""20.4 (0.02)""","""18.5 (0.02)"""
"""Mean True Positives""","""30.7 (0.04)""","""28.5 (0.03)""","""28.1 (0.03)""","""32.8 (0.04)""","""33.0 (0.04)""","""33.5 (0.04)""","""38.9 (0.05)""","""26.4 (0.03)""","""30.3 (0.04)""","""30.7 (0.04)""","""32.9 (0.04)""","""19.4 (0.02)""","""21.3 (0.03)"""


Ensambling with BalancedBaggingClassifier improved the recall score of all the models, some more than others. For example benchmark models Decision Tree and Passive Aggressive Classifier improved slightly, but not as much as other models. These classifiers are currently more prone to predict False Negatives, which is not what is needed. 

From the results, if the goal is to minimize False Negatives and maximize True Positives, the best model to use would be Logistic Regression or Ridge Classifier. These models have the highest median recall scores,- lowest amount of False Negatives, and the highest amount of True Positives.

For all the models the False Positive rate is still very high, meaning that the models are predicting the minority class too much. Prime example would be Gaussian Naive Bayes, which has a very high recall score, but also a very high False Positive rate.

Another train of thought about Gaussian Naive Bayes,- if least amount of False Negatives is the goal, then this model is pretty good at that, it however comes at a cost of high False Positive rate. But if the False Positives are considered as patients who are at higher risk of having a stroke, basically looking the same as patients who had a stroke, then this model could be useful.


Feature reduction could also help models distinguish between the two classes, and reduce the False Positive rate.


## <a id='toc6_6_'></a>[Feature Selection](#toc0_)

Feature selection will be used to reduce the number of features in the dataset. Overall this should help reduce overfitting and improve the performance of the models.

In [ ]:
print(f"Feature count after transformations: {df.shape[1]}")

Feature count after transformations: 37


### <a id='toc6_6_1_'></a>[K-Best](#toc0_)

K-Best selection can be used to check which features are the most useful in predicting the target variable. In essence, it selects the best features based on the highest statistical measure, in this case - the ANOVA F-value.

In [43]:
for i in range(1, 11):
    print(f"top {i} features: {", ".join(select_k_best(df, y_train, i).columns)}")

top 1 features: risk_score_all
top 2 features: age, risk_score_all
top 3 features: age, risk_score_all, risk_score_numeric
top 4 features: age, age_binned_senior, risk_score_all, risk_score_numeric
top 5 features: age, age_binned_senior, risk_score_all, risk_score_heart, risk_score_numeric
top 6 features: age, hypertension, age_binned_senior, risk_score_all, risk_score_heart, risk_score_numeric
top 7 features: age, hypertension, age_binned_adult, age_binned_senior, risk_score_all, risk_score_heart, risk_score_numeric
top 8 features: age, hypertension, heart_disease, age_binned_adult, age_binned_senior, risk_score_all, risk_score_heart, risk_score_numeric
top 9 features: age, hypertension, heart_disease, avg_glucose_level, age_binned_adult, age_binned_senior, risk_score_all, risk_score_heart, risk_score_numeric
top 10 features: age, hypertension, heart_disease, avg_glucose_level, age_binned_adult, age_binned_senior, risk_score_all, risk_score_no_age, risk_score_heart, risk_score_numeric

The selection shows roughly the same features that were theorized to be useful in predicting the target variable.

age along with it's binned version - age_binned_senior (age > 60) is among the most useful features in predicting the target variable. This is consistent with the exploratory data analysis and statistical inference, where it was shown that patients who had a stroke seem to be older than those who did not have a stroke.

risk_score_all - the combination of numeric features and categorical features (heart_disease, hypertension) also makes sense.

risk_score_heart - the combination of heart_disease and hypertension features.

risk_score_numeric - the combination of numeric features (age, bmi, avg_glucose_level), overall the list of features makes sense as it includes the engineered features that were created to capture non-linear relationships between features and the target variable.

In [44]:
base_model_list = build_model_list(prep_pipeline)
DillWrapper(function=evaluate_k_best, 
            model_dict=base_model_list, 
            X=X_train, 
            y=y_train, 
            scoring="recall").load(create_if_not_exist=True)

Observations recall,Random Forest,Gradient Boosting,HistGradientBoosting,Logistic Regression,SGD Classifier,Ridge Classifier,Gaussian Naive Bayes,Passive Aggressive,KNNeighbors,SVC,Linear SVC,Decision Tree,Extra Tree
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Base Mean Score""",0.004,0.026038,0.048628,0.0,0.0,0.0,0.979897,0.097359,0.012077,0.0,0.0,0.151103,0.121038
"""K-Best Mean Score""",0.095333,0.038628,0.058231,0.0,0.000513,0.0,0.979897,0.209987,0.032538,0.0015,0.0,0.164615,0.143705
"""k-best Features""",1.0,5.0,24.0,1.0,31.0,1.0,28.0,25.0,12.0,12.0,1.0,30.0,25.0


Some of the models benefited from a moderate feature reduction, with their recall scores improving slightly. 

In [45]:
DillWrapper(function=evaluate_k_best, 
            model_dict=balanced_model_list, 
            X=X_train, 
            y=y_train, 
            scoring="recall").load(create_if_not_exist=True)

Observations recall,Random Forest,Gradient Boosting,HistGradientBoosting,Logistic Regression,SGD Classifier,Ridge Classifier,Gaussian Naive Bayes,Passive Aggressive,KNNeighbors,SVC,Linear SVC,Decision Tree,Extra Tree
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Base Mean Score""",0.794449,0.78841,0.767321,0.818641,0.758385,0.817615,0.979897,0.714141,0.778449,0.765821,0.813128,0.625538,0.593949
"""K-Best Mean Score""",0.795474,0.792449,0.808526,0.819654,0.847141,0.835244,0.979897,0.772923,0.847679,0.873974,0.840269,0.625538,0.621026
"""k-best Features""",36.0,36.0,1.0,32.0,8.0,2.0,22.0,8.0,9.0,2.0,7.0,37.0,2.0


If considering models with BalancedBaggingClassifier, the models with highest gains in recall score, were the ones with the least amount of features. 

In this case SVC dominates in terms of recall score, if not considering Gaussian Naive Bayes, and only with 2 features (age, risk_score_all).

### <a id='toc6_6_2_'></a>[Sequential Feature Selection](#toc0_)

Sequential Feature Selection (SFS) can be used to select features and test the performance of the models. This is done by either adding or removing features from the dataset and checking the performance of the models. This should help to find the optimal number of features in the dataset.

In [46]:
sfs_results = DillWrapper(function=evaluate_sfs, 
                          model_dict=base_model_list, 
                          prep_pipeline=prep_pipeline,
                          X=X_train, 
                          y=y_train, 
                          direction="forward", 
                          scoring="recall").load(create_if_not_exist=True)

In [47]:
display(sfs_results[1])
display(sfs_results[0].sort("Count", descending=True).head(10))

Metric,Random Forest,Gradient Boosting,HistGradientBoosting,Logistic Regression,SGD Classifier,Ridge Classifier,Gaussian Naive Bayes,Passive Aggressive,KNNeighbors,SVC,Linear SVC,Decision Tree,Extra Tree
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Feature Count""",18,18,18,18,18,18,18,18,18,18,18,18,18


Feature,Count
str,i64
"""heart_disease""",13
"""gender_Other""",13
"""work_type_Never_worked""",12
"""work_type_children""",11
"""smoking_status_Unknown""",11
"""work_type_Self-employed""",10
"""ever_married""",10
"""work_type_Govt_job""",10
"""bmi""",9


When using Sequential Feature Selection all models used some variation of features up to count of 19(half of original). This is consistent with the K-Best selection, leading to the conclusion that there are definitely some redundant features in the dataset. 

The impact and importance of each individual feature is not as clear, because the results differ between the two approaches. For example, in K-Best selection, the most important feature is age, while in Sequential Feature Selection age does not play such a big role.

## <a id='toc6_7_'></a>[Final Model](#toc0_)

Idea for the final model is to have an ensamble of BalancedBaggingClassifier models with different classifiers, and different features. Some classifiers benefited more from feature reduction, while others less so. Three different classifiers should be enough to capture different aspects of the dataset.

Key metric consideration for tuning is recall score (reducing the amount of False Negatives), as it is the most important metric for this project.

Classifiers to use:
- Passive-Aggressive Classifier: Showed good recall and f1 scores, due to it's linear nature and ability to adjust to the data it should be a good fit for the ensemble.
- SVC: To try and capture the relationships between features and the target variable that are not linear. This classifier benefited the most from feature reduction and BalancedBaggingClassifier usage.
- HistGradientBoosting: A non-linear classifier that should capture non-linear relationships between features and the target variable.

Hyper parameter tuning pipeline:
- Feature transformations
- Feature engineering
- Feature selection (k-best)
- BalancedBaggingClassifier with selected classifiers as base estimators
- Class weights for each classifier - balanced, to try and handle the imbalance in the dataset

Best parameters found by using GridSearchCV

(Couple of other classifiers were tested, to try and evaluate the best model)


In [48]:
final_model_dict = build_final_models_dict()
final_model_dict["SVC"]["model"]

ModelWrapper(model=BalancedBaggingClassifier(estimator=SVC(class_weight='balanced',
                                                           random_state=42),
                                             random_state=42,
                                             replacement=True),
             prep_pipeline=Pipeline(steps=[('preprocessor',
                                            FeaturePreprocessor(column_pipelines={'Residence_type': Pipeline(steps=[('cat_imputer',
                                                                                                                     NumericImputer(adj_missing_value=False,
                                                                                                                                    imputer='simple',
                                                                                                                                    strategy='most_frequent')),
                                                                                                                    (...
                                                                                  scaler_obj=NumericScaler())),
                                                            ('risk_score_heart',
                                                             BasicFeatureCombiner(columns=['heart_disease',
                                                                                           'hypertension'],
                                                                                  new_feature_name='risk_score_heart',
                                                                                  scaler_obj=NumericScaler())),
                                                            ('risk_score_numeric',
                                                             BasicFeatureCombiner(columns=['bmi',
                                                                                           'avg_glucose_level',
                                                                                           'age'],
                                                                                  new_feature_name='risk_score_numeric',
                                                                                  scaler_obj=NumericScaler()))])),
                                           ('k_best', SelectKBest(k=2))]))

In [49]:
tune_result = DillWrapper(function=hypertune_models,
                          model_dict=build_final_models_dict(),
                          X=X_train,
                          y=y_train,
                          scoring="recall",
                          verbose=True).load(create_if_not_exist=True)

In [ ]:
model_dict = {}
for name, data in tune_result.items():
    print(f"Model: {name}, "
          f"avg. recall: {data['score']}, "
          f"k-features: {data['params']['prep_pipeline__k_best__k']}")
    model_dict[name] = data["model"]
display(model_score_matrix(model_dict, X_train, y_train, scoring_metric="recall")[1])
display(evaluate_confusion_matrix(model_dict, X_train, y_train, n_splits=5, n_repeats=2))

Model: SVC, avg. recall: 0.9547435897435899, k-features: 2
Model: RidgeClassifier, avg. recall: 0.8644871794871796, k-features: 16
Model: HistGradientBoostingClassifier, avg. recall: 0.8843589743589744, k-features: 1
Model: Perceptron, avg. recall: 0.8746153846153846, k-features: 4
Model: PassiveAggressiveClassifier, avg. recall: 0.99, k-features: 2
Model: DecisionTreeClassifier, avg. recall: 0.8946153846153846, k-features: 2


Observation,SVC,RidgeClassifier,HistGradientBoostingClassifier,Perceptron,PassiveAggressiveClassifier,DecisionTreeClassifier
str,str,str,str,str,str,str
"""Mean True Negatives""","""444.3 (0.54)""","""548.9 (0.67)""","""483.8 (0.59)""","""538.3 (0.66)""","""342.9 (0.42)""","""520.4 (0.64)"""
"""Mean False Positives""","""333.5 (0.41)""","""228.9 (0.28)""","""294.0 (0.36)""","""239.5 (0.29)""","""434.9 (0.53)""","""257.4 (0.31)"""
"""Mean False Negatives""","""1.7 (0.0)""","""6.6 (0.01)""","""4.7 (0.01)""","""7.1 (0.01)""","""0.4 (0.0)""","""5.0 (0.01)"""
"""Mean True Positives""","""38.1 (0.05)""","""33.2 (0.04)""","""35.1 (0.04)""","""32.7 (0.04)""","""39.4 (0.05)""","""34.8 (0.04)"""


### <a id='toc6_7_1_'></a>[Voting Classifier](#toc0_)

Voting Classifier will be used to combine hypertuned models into one model. This should help to improve the performance of the models and reduce overfitting.


In [ ]:
voting_model = DillWrapper(function=VotingClassifier,
                           estimators=[("SVC", model_dict["SVC"]),
                                       ("PassiveAggressive", model_dict["PassiveAggressiveClassifier"]),
                                       ("DecisionTreeClassifier", model_dict["DecisionTreeClassifier"])],
                           voting="soft",
                           n_jobs=-1).load(create_if_not_exist=True)

display(model_score_matrix({"VotingClassifier": voting_model}, X_train, y_train, scoring_metric="recall")[1])
display(evaluate_confusion_matrix({"VotingClassifier": voting_model}, X_train, y_train, n_splits=5, n_repeats=2))


Observation,VotingClassifier
str,str
"""Mean True Negatives""","""423.7 (0.52)"""
"""Mean False Positives""","""354.1 (0.43)"""
"""Mean False Negatives""","""1.5 (0.0)"""
"""Mean True Positives""","""38.3 (0.05)"""


# <a id='toc7_'></a>[**Final Model - Test Set**](#toc0_)

The final model is soft Voting Classifier, which combines hypertuned BalancedBaggingClassifier of Passive-Aggressive Classifier, SVC, and Decision Tree. 

The model was trained and evaluated on the training set.

The model will now be evaluated on the testing set to see how well it generalizes to new data.

In [52]:
model = voting_model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

confusion = confusion_matrix(y_test, y_pred)
display(px.imshow(confusion, title="Confusion Matrix", labels=dict(x="Predicted", y="True"), color_continuous_scale="Blues", text_auto=True))

              precision    recall  f1-score   support

           0       0.99      0.56      0.72       972
           1       0.09      0.86      0.17        50

    accuracy                           0.58      1022
   macro avg       0.54      0.71      0.44      1022
weighted avg       0.94      0.58      0.69      1022



In [53]:
plot_precision_recall_curve(model, X_test, y_test)

While the accuracy of the model is not the best (0.58), the recall score is high (0.86), which means that the model avoids predicting negative class (not having a stroke) leading to lower amount of False Negatives, which is crucial in this case. It is better to predict that a patient will have a stroke and be wrong, than to predict that a patient will not have a stroke and be wrong.

Ofcourse the high amount of False Positives is not ideal, but in this case it is more important to minimize False Negatives.

# <a id='toc8_'></a>[**Summary**](#toc0_)

Steps taken to achieve results:
- Data exploration:
    - Found problems with data itself,- missing values, imbalance, data types.
    - Found data relationships,- age is one of the significant features in predicting the target variable.
    - Correlation - not all features are correlated to the target feature, and some features are correlated to each other.
- Statistical inference:
    - Found statistical differences between patients who had a stroke and those who did not have a stroke.
- Machine learning models:
    - Created preprocessing pipeline to handle missing values, normalize data, and convert data types.
    - Created preprocessing step to engineer new features.
    - Tried different models to see which one performs the best. Determined that recall score is the most important metric for model selection.
    - Tried to fix data imbalance using BalancedBaggingClassifier.
    - Checked a couple of feature selection methods to reduce the number of features in the dataset. Added feature selection(k-best) to the preprocessing pipeline.
    - Hyperparameter tuned a couple of classifiers to use in the final model.
    - Created a final model using Voting Classifier, which combines hypertuned BalancedBaggingClassifier of Passive-Aggressive Classifier, SVC, and Decision Tree.


Final model:
- Soft Voting Classifier
    - BalancedBaggingClassifier with base estimators: Passive-Aggressive Classifier, SVC, Decision Tree
    - Class weights for each classifier - balanced
- Fit model hash: 5edda379ff46f6d1cdffde234e0bfa6a

Results:
- Accuracy: 0.58
- Recall: 0.86
- F1: 0.17
- Trade-off between False Positives and False Negatives, with the model having a high recall score, the number of False Negatives is very low, but the number of False Positives is high.


Suggestions for improvement:
- More feature engineering could be done to capture non-linear relationships between features and the target variable.
    - Perhaps clustering could be used to group patients into different categories.
    - Some of the features in EDA and statistical inference suggested having an impact on the target variable, but none of the final models used them. Perhaps these features were mis-utilized or not used correctly e.g. smoking_status was one hot encoded, but none of the models used it.
- SHAP or Lime usage to explain the model predictions.
    - Due to custom transformers and pipelines, there were numerous complications in using SHAP or Lime to explain the model predictions. This could be improved by using scikit-learn pipelines more effectively.
- Perhaps focus on different metrics, like precision, could be used to improve the model.
- Custom transformers and pipelines at some points created more problems than they solved. Perhaps a more streamlined approach could be used to create the final model.


